# Assignment

## 1. Go to the [eurostat](http://ec.europa.eu/eurostat/data/database) website and try to find a dataset that includes the european unemployment rates at a recent date.

Use this data to build a [Choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) which shows the unemployment rate in Europe at a country level. Think about [the colors you use](https://carto.com/academy/courses/intermediate-design/choose-colors-1/), how you decided to [split the intervals into data classes](http://gisgeography.com/choropleth-maps-data-classification/) or which interactions you could add in order to make the visualization intuitive and expressive. Compare Switzerland's unemployment rate to that of the rest of Europe.

In [1]:
import os
import pandas as pd
import json
import folium
import topojson

In [50]:
assert(folium.__version__ == '0.5.0') # do we have the right version of folium?

We pick the `tsdec450.tsv` dataset, found here:

_Populations and social conditions_ -> _Labour markets_ -> _Employment and unemployment_ -> _Unemployment_ -> _Total unemployment rate_

This dataset contains yearly unemployment data for all countries of interest.

Note: the dataset is in Tab-Separated Values (TSV) format, which we can parse simply by setting a `'\t'` separator.

In [48]:
# Import dataset
unemployment_eu = pd.DataFrame.from_csv('tsdec450.tsv', sep='\t')
unemployment_eu = unemployment_eu.rename(columns=lambda x: x.strip()) # trim whitespaces in headers

# Let's see what this thing looks like
unemployment_eu

1990    1991   1992   1993   1994    1995   1996  \
age,unit,sex,geo\time                                                       
TOTAL,PC_ACT,T,AT          :       :      :      :    4.3     4.2    4.7    
TOTAL,PC_ACT,T,BE        6.6     6.4    7.1    8.6    9.8     9.7    9.5    
TOTAL,PC_ACT,T,BG          :       :      :      :      :       :      :    
TOTAL,PC_ACT,T,CY          :       :      :      :      :       :      :    
TOTAL,PC_ACT,T,CZ          :       :      :    4.3    4.3     4.0    3.9    
TOTAL,PC_ACT,T,DE          :     5.5    6.6    7.8    8.4     8.2    8.9    
TOTAL,PC_ACT,T,DK        7.2     7.9    8.6    9.6    7.7     6.7    6.3    
TOTAL,PC_ACT,T,EA18        :       :      :      :      :       :      :    
TOTAL,PC_ACT,T,EA19        :       :      :      :      :       :      :    
TOTAL,PC_ACT,T,EE          :       :      :      :      :       :      :    
TOTAL,PC_ACT,T,EL          :       :      :      :      :       :      :    
TOTAL,PC_ACT,T,ES       15.5    15.5   17.0   20.8   22.0   20.7 i  19.9    
TOTAL,PC_ACT,T,EU27        :       :      :      :      :       :      :    
TOTAL,PC_ACT,T,EU28        :       :      :      :      :       :      :    
TOTAL,PC_ACT,T,FI        3.2     6.6   11.7   16.3   16.6    15.4   14.6    
TOTAL,PC_ACT,T,FR        7.9     8.1    9.0   10.1   10.4   10.2 i  10.5    
TOTAL,PC_ACT,T,HR          :       :      :      :      :       :      :    
TOTAL,PC_ACT,T,HU          :       :      :      :      :       :    9.9    
TOTAL,PC_ACT,T,IE       13.4    14.7   15.4   15.6   14.3    12.3   11.7    
TOTAL,PC_ACT,T,IS          :       :      :      :      :       :      :    
TOTAL,PC_ACT,T,IT       8.9 p   8.5 p   8.8    9.7   10.6    11.2   11.2    
TOTAL,PC_ACT,T,JP        2.1     2.1    2.2    2.5    2.9     3.1    3.4    
TOTAL,PC_ACT,T,LT          :       :      :      :      :       :      :    
TOTAL,PC_ACT,T,LU        1.7     1.6    2.1    2.6    3.2     2.9    2.9    
TOTAL,PC_ACT,T,LV          :       :      :      :      :       :      :    
TOTAL,PC_ACT,T,MT          :       :      :      :      :       :      :    
TOTAL,PC_ACT,T,NL        6.1     5.7    5.7    6.5    7.2     8.3    7.7    
TOTAL,PC_ACT,T,NO        5.2     5.5    5.9    6.0    5.4    4.9 b   4.7    
TOTAL,PC_ACT,T,PL          :       :      :      :      :       :      :    
TOTAL,PC_ACT,T,PT        5.6     5.0    5.0    6.3    7.6     7.9    8.0    
...                       ...     ...    ...    ...    ...     ...    ...   
TOTAL,THS_PER,T,EA18       :       :      :      :      :       :      :    
TOTAL,THS_PER,T,EA19       :       :      :      :      :       :      :    
TOTAL,THS_PER,T,EE         :       :      :      :      :       :      :    
TOTAL,THS_PER,T,EL         :       :      :      :      :       :      :    
TOTAL,THS_PER,T,ES      2297    2313   2602   3230   3455   3279 i  3192    
TOTAL,THS_PER,T,EU27       :       :      :      :      :       :      :    
TOTAL,THS_PER,T,EU28       :       :      :      :      :       :      :    
TOTAL,THS_PER,T,FI        82     168    292    405    408     382    363    
TOTAL,THS_PER,T,FR      2007    2074   2314   2591   2696   2649 i  2763    
TOTAL,THS_PER,T,HR         :       :      :      :      :       :      :    
TOTAL,THS_PER,T,HU         :       :      :      :      :     391    395    
TOTAL,THS_PER,T,IE       176     197    209    216    203     177    174    
TOTAL,THS_PER,T,IS         :       :      :      :      :       :      :    
TOTAL,THS_PER,T,IT     2084 p  2023 p  2056   2296   2493    2616   2619    
TOTAL,THS_PER,T,JP      1341    1363   1423   1657   1921    2099   2250    
TOTAL,THS_PER,T,LT         :       :      :      :      :       :      :    
TOTAL,THS_PER,T,LU         3       3      3      4      5       5      5    
TOTAL,THS_PER,T,LV         :       :      :      :      :       :      :    
TOTAL,THS_PER,T,MT         :       :      :      :      :       7      8    
TOTAL,THS_PER,T,NL   

As we can see, this table contains unemployment as a percentage of the population in age of working (PC_POP) and as a percentage of the active workforce (PC_ACT). We pick the latter (unemployment as a percentage of the active workforce or PC_ACT) for the rest of this assignment. 

For mapping purposes, we simply need to extract the rows of interest (PC_ACT) and project on the latest year (2016). Once done, we extract the country initials (last 2 letters) from the index column and we have the data we want.

In [53]:
recent_unemployment_eu = unemployment_eu[unemployment_eu.index.str.match(".*PC_ACT.*")]['2016'].reset_index()
recent_unemployment_eu.columns = ['country', 'unemployment']
recent_unemployment_eu['country'] = recent_unemployment_eu['country'].str.extract('([A-z]{2})$', expand=False)
recent_unemployment_eu = recent_unemployment_eu.dropna()
recent_unemployment_eu

country  unemployment
0       AT           6.0
1       BE           7.8
2       BG           7.6
3       CY          13.0
4       CZ           4.0
5       DE           4.1
6       DK           6.2
9       EE           6.8
10      EL          23.6
11      ES          19.6
14      FI           8.8
15      FR          10.1
16      HR          13.4
17      HU           5.1
18      IE           7.9
19      IS           3.0
20      IT          11.7
21      JP           3.1
22      LT           7.9
23      LU           6.3
24      LV           9.6
25      MT           4.7
26      NL           6.0
27      NO           4.7
28      PL           6.2
29      PT          11.2
30      RO           5.9
31      SE           6.9
32      SI           8.0
33      SK           9.7
34      TR          10.9
35      UK           4.8
36      US           4.9

We now plot the unemployment rate on the map. We use a yellow-orange-red color palette with the default interval splits (yellow from 3%, red at 24%).

In [57]:
# Get the topology
topo = json.load(open("topojson/europe.topojson.json"))

# Center the map and scale it (zoom = 4)
europe_coordinates = [53, 9]
topo_map = folium.Map(location=europe_coordinates, zoom_start=4)
topo_map.choropleth(topo, topojson='objects.europe',
             data=recent_unemployment_eu,
             columns=['country', 'unemployment'], 
             key_on='feature.id',
             fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Unemployment rate (% of the active workforce)')

# Let's take a look...
topo_map

As we can see, the UK is doing pretty well (though that may change after Brexit), Germany not too bad, France and Italy are leaning towards the orange, and Spain is trailing at almost 20%. 

## 2. Go to the [amstat](https://www.amstat.ch) website to find a dataset that includes the unemployment rates in Switzerland at a recent date.

   > *HINT* Go to the `details` tab to find the raw data you need. If you do not speak French, German or Italian, think of using free translation services to navigate your way through. 

   Use this data to build another Choropleth map, this time showing the unemployment rate at the level of swiss cantons. Again, try to make the map as expressive as possible, and comment on the trends you observe.

   The Swiss Confederation defines the rates you have just plotted as the number of people looking for a job divided by the size of the active population (scaled by 100). This is surely a valid choice, but as we discussed one could argue for a different categorization.

   Copy the map you have just created, but this time don't count in your statistics people who already have a job and are looking for a new one. How do your observations change ? You can repeat this with different choices of categories to see how selecting different metrics can lead to different interpretations of the same data.



## 3. Use the [amstat](https://www.amstat.ch) website again to find a dataset that includes the unemployment rates in Switzerland at recent date, this time making a distinction between *Swiss* and *foreign* workers.

   The Economic Secretary (SECO) releases [a monthly report](https://www.seco.admin.ch/seco/fr/home/Arbeit/Arbeitslosenversicherung/arbeitslosenzahlen.html) on the state of the employment market. In the latest report (September 2017), it is noted that there is a discrepancy between the unemployment rates for *foreign* (`5.1%`) and *Swiss* (`2.2%`) workers. 

   Show the difference in unemployment rates between the two categories in each canton on a Choropleth map (*hint* The easy way is to show two separate maps, but can you think of something better ?). Where are the differences most visible ? Why do you think that is ?

   Now let's refine the analysis by adding the differences between age groups. As you may have guessed it is nearly impossible to plot so many variables on a map. Make a bar plot, which is a better suited visualization tool for this type of multivariate data.



## 4. *BONUS*: using the map you have just built, and the geographical information contained in it, could you give a *rough estimate* of the difference in unemployment rates between the areas divided by the [Röstigraben](https://en.wikipedia.org/wiki/R%C3%B6stigraben)?